In [1]:
import os
import random

import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

# 시드 고정
seed_everything(42)

In [3]:
# train data 가져오기

df2 = pd.read_csv('K-League-data2.csv').drop(columns='Unnamed: 0')
print(df2.shape)
df2.head(1)

(666, 99)


,Rnd.,대회,홈 득점,홈 도움,홈 슈팅,홈 유효 슈팅,홈 블락된슈팅,홈 벗어난슈팅,홈 PA내 슈팅,홈 PA외 슈팅,...,원정 피파울,원정 경고,원정 퇴장,원정 팀,원정 경기결과,원정 소속 선수 수,원정 평균 나이,원정 용병 수,원정 평균 가치,원정 총 가치
0,1,대구vs수원FC,1,1,13,4,2,7,5,8,...,13,3,0,suwonFC,D,36,25.6,5,350.0,12.6


In [4]:
# train data 처리

X = df2.drop(columns=['대회', '홈 경기결과', '원정 경기결과']) # 필요 없거나 중복되는 열 제거
X = pd.get_dummies(X, columns=['홈 팀', '원정 팀'], drop_first=False) # 문자 숫자로 바꾸기

y = df2['홈 경기결과']  # 홈 팀 기준으로 승부 예측. (홈 승? >> 원정 패, 홈 패? >> 원정 승리) // 원정 경기결과를 넣어도 예측 결과 동일

# 경기 결과 (W, D, L)를 숫자로 
le = LabelEncoder()
y = le.fit_transform(y)

# train/test 분리. 데이터셋의 개수가 작아서 9:1로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# scaler = StandardScaler()  
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)
# X.columns



(599, 122) (67, 122)
(599,) (67,)


In [5]:
# test data 가져오기

test_df = pd.read_csv('K-League-Test3536.csv')
predict_df = (test_df[['Rnd.', '홈 팀','원정 팀']])

test_df = test_df.drop(columns=['Unnamed: 0', '대회', '홈 경기결과', '원정 경기결과'])

test_df = pd.get_dummies(test_df, columns=['홈 팀', '원정 팀'], drop_first=False)

test_df.shape #(12,114)

# 결과확인을 위한 df
predict_df

,Rnd.,홈 팀,원정 팀
0,36,ulsan,pohang
1,36,suwonFC,suwon
2,36,daegu,gwangju
3,36,jeju,seoul
4,36,daejeon,gangwon
5,36,incheon,jeonbuk
6,37,jeonbuk,gwangju
7,37,gangwon,suwonFC
8,37,incheon,ulsan
9,37,seoul,suwon


### 개수가 122개 / 114개로 안맞음

- 왜? ) train에는 21시즌 or 22시즌에 있었던 성남, 김천 데이터가 포함되어있음.
또한 test에는 2 라운드밖에 없으니 없는 데이터가 있음 

In [6]:
# 김천/성남은 2023시즌에 없기 때문에 드랍.
# 홈-광주, 원정-인천, 원정-제주, 홈-수원FC는 36,37라운드에 포함되지 않아 드랍.

drop_list = ['홈 팀_gimcheon', '원정 팀_gimcheon',  '홈 팀_seongnam','원정 팀_seongnam', 
            '홈 팀_gwangju', '원정 팀_incheon', '원정 팀_jeju', '홈 팀_suwon']

In [7]:
# train_data 다시 처리

X = df2.drop(columns=['대회', '홈 경기결과', '원정 경기결과']) # 필요 없거나 중복되는 열 제거
X = pd.get_dummies(X, columns=['홈 팀', '원정 팀'], drop_first=False) # 문자 숫자로 바꾸기

X = X.drop(columns=drop_list)


y = df2['홈 경기결과']  
print(y.value_counts())

# 경기 결과 (W, D, L)를 숫자로 
le = LabelEncoder()
y = le.fit_transform(y)
# print(y)

# train/test 분리. 데이터셋의 개수가 작아서 9:1로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# scaler = StandardScaler()  
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)  # 114개의 열로 맞춤

W    260
L    209
D    197
Name: 홈 경기결과, dtype: int64
(599, 114) (67, 114)
(599,) (67,)


# 변수 선택 전 성능이 가장 좋았던 모델만 사용하여 예측

In [15]:
predict_map = {0:'D', 1: "L", 2:'W'}  # 0,1,2로 바꾼걸 다시 DLW로 매칭

- SVM


In [16]:
from sklearn import svm

# train data로 모델 학습
model = svm.SVC(random_state=42, kernel='linear')
model.fit(X_train_scaled, y_train)

# test data로 predict
y_pred = model.predict(test_df)
print(y_pred)

pred_list = []
for i in y_pred:
    pred_list.append(predict_map[i])
predict_df['홈 경기결과'] = pred_list
predict_df

[0 1 0 0 0 0 0 0 0 0 1 0]


c:\Users\HONG\miniconda3\envs\data\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


,Rnd.,홈 팀,원정 팀,홈 경기결과
0,36,ulsan,pohang,D
1,36,suwonFC,suwon,L
2,36,daegu,gwangju,D
3,36,jeju,seoul,D
4,36,daejeon,gangwon,D
5,36,incheon,jeonbuk,D
6,37,jeonbuk,gwangju,D
7,37,gangwon,suwonFC,D
8,37,incheon,ulsan,D
9,37,seoul,suwon,D


SVM : x o o o x o, x x x x x x

- GradientBoosting

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=42, max_depth=3, n_estimators=500, )
model.fit(X_train_scaled, y_train)

y_pred = model.predict(test_df)
print(y_pred)

pred_list = []
for i in y_pred:
    pred_list.append(predict_map[i])
predict_df['홈 경기결과'] = pred_list
predict_df

[0 0 0 0 1 0 0 0 0 1 1 0]


c:\Users\HONG\miniconda3\envs\data\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


,Rnd.,홈 팀,원정 팀,홈 경기결과
0,36,ulsan,pohang,D
1,36,suwonFC,suwon,D
2,36,daegu,gwangju,D
3,36,jeju,seoul,D
4,36,daejeon,gangwon,L
5,36,incheon,jeonbuk,D
6,37,jeonbuk,gwangju,D
7,37,gangwon,suwonFC,D
8,37,incheon,ulsan,D
9,37,seoul,suwon,L


GradientBoost : x x o o o o , x x x o o x

- XGBoost

In [18]:
from xgboost import XGBClassifier


model = XGBClassifier()

model.fit(X_train_scaled, y_train)

y_pred = model.predict(test_df)
print(y_pred)

pred_list = []
for i in y_pred:
    pred_list.append(predict_map[i])
predict_df['홈 경기결과'] = pred_list
predict_df

[0 0 1 1 1 1 2 2 2 1 1 2]


,Rnd.,홈 팀,원정 팀,홈 경기결과
0,36,ulsan,pohang,D
1,36,suwonFC,suwon,D
2,36,daegu,gwangju,L
3,36,jeju,seoul,L
4,36,daejeon,gangwon,L
5,36,incheon,jeonbuk,L
6,37,jeonbuk,gwangju,W
7,37,gangwon,suwonFC,W
8,37,incheon,ulsan,W
9,37,seoul,suwon,L


XGBoost : x x x x o x, o o o o o o

- CatBoost

In [20]:
from catboost import CatBoostClassifier


model = CatBoostClassifier(random_seed=42, metric_period=100, depth=4)

model.fit(X_train_scaled, y_train)

y_pred = model.predict(test_df)
print(y_pred)

y_pred = y_pred.reshape(-1)
pred_list = []
for i in y_pred:
    pred_list.append(predict_map[i])
predict_df['홈 경기결과'] = pred_list
predict_df

Learning rate set to 0.077085
0:	learn: 1.0291243	total: 6.3ms	remaining: 6.29s
100:	learn: 0.1718829	total: 308ms	remaining: 2.74s
200:	learn: 0.1019578	total: 638ms	remaining: 2.53s
300:	learn: 0.0754662	total: 939ms	remaining: 2.18s
400:	learn: 0.0533483	total: 1.23s	remaining: 1.83s
500:	learn: 0.0425894	total: 1.51s	remaining: 1.51s
600:	learn: 0.0342938	total: 1.78s	remaining: 1.18s
700:	learn: 0.0289653	total: 2.06s	remaining: 880ms
800:	learn: 0.0243735	total: 2.33s	remaining: 579ms
900:	learn: 0.0211503	total: 2.59s	remaining: 285ms
999:	learn: 0.0189985	total: 2.86s	remaining: 0us
[[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [2]
 [2]
 [0]
 [1]
 [1]
 [2]]


,Rnd.,홈 팀,원정 팀,홈 경기결과
0,36,ulsan,pohang,D
1,36,suwonFC,suwon,D
2,36,daegu,gwangju,D
3,36,jeju,seoul,D
4,36,daejeon,gangwon,L
5,36,incheon,jeonbuk,D
6,37,jeonbuk,gwangju,W
7,37,gangwon,suwonFC,W
8,37,incheon,ulsan,D
9,37,seoul,suwon,L


CatBoost : x x o o o o, o o x o o o

## CatBoost가 가장 예측을 잘 하였다.

다음으로 XGBoost, GradientBoost, SVM 순